# *Imports*

In [16]:
from composable import pipeable
from composable.strict import map, filter
import composable_records as rec
import composable_tuples as tup
from composable_glob import glob
from composable_origami import fold
from composable_utility import with_open, get
import pandas as pd

# **Paths and course-related functions**

In [17]:
# **Paths and course-related functions**
paths = (
    "./attendance_example_fixed_width/*/*.txt"
    >> glob(recursive=True)
)

In [18]:
normalize_path = lambda path: path.replace("\\", "/")
get_course = lambda path: normalize_path(path).split("/")[-2]
get_section = lambda path: get_course(path)[-2:]
get_type = lambda path: "Attendance" if "Attendance Quiz" in path else "Practice Quiz"

# **Processing files and lines**

In [19]:

paths_and_lines = [
    {
        "course": get_course(path),
        "section": get_section(path),
        "type": get_type(path),
        "lines": with_open(lambda f: f.readlines(), path),
    }
    for path in paths
]

# **Header and column definition**

In [20]:

header = [
    ("Org Defined ID", 9), ("UserName", 9), ("FirstName", 12), ("LastName", 12),
    ("Attempt #", 3), ("Score", 3), ("Out Of", 3), ("Attempt_Start", 20),
    ("Attempt_End", 20), ("Percent", 4)
]

column_names = [col[0] for col in header]
column_widths = [col[1] for col in header]

# **Column positions calculation based on widths**

In [21]:
positions = [sum(column_widths[:i]) for i in range(len(column_widths) + 1)]

# **Parse records from lines**

In [22]:

parsed_records = [
    {
        "course": record["course"],
        "section": record["section"],
        "type": record["type"],
        **dict(zip(column_names, [line[positions[i]:positions[i + 1]].strip() for i in range(len(column_widths))]))
    }
    for record in paths_and_lines
    for line in record["lines"]
]

# **Splitting Attempt Start and End into Date and Time**

In [23]:

for record in parsed_records:
    record["Attempt_Start_Date"], record["Attempt_Start_Time"] = record["Attempt_Start"].split(" ") if " " in record["Attempt_Start"] else ("", "")
    record["Attempt_End_Date"], record["Attempt_End_Time"] = record["Attempt_End"].split(" ") if " " in record["Attempt_End"] else ("", "")


# **Convert parsed records into a DataFrame**

In [24]:
df = pd.DataFrame(parsed_records)

# **Save merged data to CSV**

In [25]:

output_csv = "merged_attendance_practice.csv"
df.to_csv(output_csv, index=False)

print(f" Successfully merged and saved as {output_csv}")

 Successfully merged and saved as merged_attendance_practice.csv
